In [11]:
#!pip install xgboost

In [2]:
import os
import torch


descriptor_dir = "data/ICIAR2018_BACH_Challenge/Photos/A1_brightness_aug_prepost_norm_3norm_effnet_descriptors"

def check_for_nans(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if file is a PyTorch tensor file
            #if file.endswith(".pt"):
            # Load PyTorch tensor from file
            tensor = torch.load(os.path.join(root, file))
            # Check if tensor contains NaN values
            if torch.isnan(tensor).any():
                print(f"File {os.path.join(root, file)} contains NaN values")
                
                
check_for_nans(f"{descriptor_dir}/Normal")

In [3]:
import random

random.seed(666)

# Generate 25 random numbers between 1 and 100
numbers = [random.randint(1, 100) for _ in range(25)]

# Pad each number with leading zeros to make them all the same length
test_numbers = [str(num).zfill(3) for num in numbers]

print(test_numbers)

['059', '049', '056', '037', '065', '002', '071', '071', '100', '092', '043', '016', '007', '019', '050', '081', '060', '075', '032', '003', '015', '035', '013', '039', '088']


In [12]:
# /data/ICIAR2018_BACH_Challenge/Photos/A1_brightness_aug_prepost_norm_3norm_effnet_descriptors
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import torch
from pathlib import Path

import os
import seaborn as sns

descriptor_dir = "data/ICIAR2018_BACH_Challenge/Photos/A1_brightness_aug_prepost_norm_3norm_effnet_descriptors"

# Read input images and assign labels based on folder names
print(os.listdir(descriptor_dir))

#Capture training data and labels into respective lists
train_images = []
train_labels = [] 
test_images = []
test_labels = []

labels = {"Invasive":1, "Benign": 0, "Normal": 0, "InSitu":1}
# labels = {"Invasive":0, "Benign": 1, "Normal": 2, "InSitu":3}

for directory_path in glob.glob(f"{descriptor_dir}/*"):
    if directory_path.split(".")[-1] == "py": continue
    label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*")):
        descriptor_tensor = torch.load(img_path).flatten()
        orig_file_name = Path(img_path).name.split("_")[0]
        if any(num in orig_file_name for num in test_numbers): # if part of test set. Have to do it like this because there are 15 augmentations of each img
            if "10199999809265137" in img_path: # this should just be a 1 if encodings included a brightness of factor 1 which it should but doesnt
#                 print(img_path)
                test_images.append(descriptor_tensor)
                test_labels.append(labels[label])
        else: # if not part of test set 
            train_images.append(descriptor_tensor)
            train_labels.append(labels[label])

#Convert lists to arrays        
train_images = torch.stack(train_images, dim=0)
train_labels = torch.tensor(train_labels)

test_images = torch.stack(test_images, dim=0)
test_labels = torch.tensor(test_labels)

# print(images)
# print(labels)

['Invasive', 'GENERATOR_SCRIPT.py', 'Benign', 'InSitu', 'Normal']


In [13]:
#XGBOOST
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(train_images, train_labels) #For sklearn no one hot encoding

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [14]:
#Now predict using the trained RF model. 
prediction = model.predict(test_images)
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction))

Accuracy =  0.90625
